In [167]:
import redis
from elasticsearch import Elasticsearch
from elasticsearch.helpers import streaming_bulk,bulk,parallel_bulk



In [252]:
# redis DB with edge info
r0 = redis.StrictRedis(host="ec2-52-39-225-153.us-west-2.compute.amazonaws.com", port=6379, db=0)

# redis DB with userID: user name
r1 = redis.StrictRedis(host="ec2-52-39-225-153.us-west-2.compute.amazonaws.com", port=6379, db=1)


In [254]:
r1.get("286542")

In [169]:
es = Elasticsearch(
   ['52.41.91.188:9200', '52.41.104.252:9200','52.26.39.161:9200'],
   sniff_on_start=False,
   sniff_on_connection_fail=False,
   #sniffer_timeout=60
   )
   

In [170]:
def get_friend(vertex):
    return r0.smembers(vertex)

In [141]:
len(get_friend("343071"))

52

In [161]:
def get_degree(vertex):
    return len(r0.smembers(vertex))
    

In [143]:
get_degree("343071")

52

In [162]:
def get_2nd_friend(vertex):
    second_degree = set()
    friends = get_friend(vertex)
    for s in friends:
        second_degree = second_degree.union(r0.smembers(s))
    return second_degree-friends
    

In [145]:
len(get_2nd_friend("343071"))

846

In [244]:
def get_distance(r,a,b,updated_time): # if updated_time ='n/a', if there is one record in ES, assume it is not current, 
    # so the record is old, no new conenction, distance  = 1
    if test_new_connection(a,b,updated_time) == False:
        return 1
 #   if a in r.smembers(b) or b in r.smembers(a):
 #       return 1
    else:
        r.srem(a,b)
        
        if r.smembers(a).isdisjoint(r.smembers(b)) !=True:
            r.sadd(a,b)
            return 2
        else:

            second_degree = set()
            for s in r.smembers(a):
                second_degree = second_degree.union(r.smembers(s))
            #print second_degree
            #print r.smembers(b)
            #print second_degree & r.smembers(b)
            if second_degree.isdisjoint(r.smembers(b)):
                r.sadd(a,b)
                return 0
            else:
                r.sadd(a,b)
                return 3
        
        

In [249]:
get_distance(r0,"343071","139608",'2015-08-11T20:16:12Z')

2

In [148]:
from collections import Counter
def friend_recommend(r,a):
    
    second_degree = []
    friends = get_friend(a)
    for s in friends:
        #print r.smembers(s)
        second_degree.extend(list(r.smembers(s)))
    recommend_list = []
    newcounter = Counter(second_degree)
    del newcounter[a]
    for i in friends:
        del newcounter[i]

    return newcounter.most_common(5)




In [149]:
friend_recommend(r0,"2715857")

[('782786', 3), ('1163921', 3), ('1948462', 3), ('1525963', 3), ('3248945', 3)]

In [241]:

def test_new_connection(a,b,updated_time): # test if this is the first connection between the two users.
    query ={ 
    "size":2, 
       "query" : {

                "bool" : {
                  "must" : [
                     { "term" : {"actor.id" : a}}, 
                     { "term" : {"transactions.target.id" : b}} 
                  ]

             }
          }
    }
    res = es.search(index="venmo",doc_type="transaction", body=query)
    if res['hits']['total'] == 0: 
    # current record has not been indexed by ES and no previous record
        return True
    elif res['hits']['total'] == 1 :
        if res['hits']['hits'][0]['_source']['updated_time'] == updated_time:
              # only current record in ES 
            return True
        
        else:
            # older 1 record in ES
            return False
      

    else:
         # more than 1 records in ES
        return False


In [243]:
test_new_connection('343071','139608',"2015-08-11T20:16:12Z")

True

In [250]:
import json

def get_recent_transactions(id): # for specific user, also with distance between the pair of users.
    query ={ 
    "size":50, 
       "query" : {
          "constant_score" : { 
             "filter" : {
                "bool" : {
                  "should" : [
                     { "term" : {"actor.id" : id}}, 
                     { "term" : {"transactions.target.id" : id}} 
                  ]
                  }
               }
             }
          },
          "sort": { "created_time": { "order": "desc" }}
    }
    res = es.search(index="venmo",doc_type="transaction", body=query)
    transactions = []
    for hit in res['hits']['hits']:
        payment_id = hit['_source']['payment_id']
        time = hit['_source']['updated_time']
        message = hit['_source']['message']
        actor_id = hit['_source']['actor']['id']
        actor_name = hit['_source']['actor']['name']
        target_id = hit['_source']['transactions'][0]['target']['id']
        target_name = hit['_source']['transactions'][0]['target']['name']
        distance = get_distance(r0,actor_id,target_id,time) # add time stamp
        transactions.append([payment_id, time,message,actor_id,actor_name,target_id,target_name,distance])
    return transactions
    

In [222]:
es

<Elasticsearch([{u'host': u'52.41.91.188', u'port': 9200}, {u'host': u'52.41.104.252', u'port': 9200}, {u'host': u'52.26.39.161', u'port': 9200}])>

In [251]:
get_recent_transactions("1061933")

[[44405045,
  u'2015-03-08T22:22:21Z',
  u'\U0001f6a3\U0001f603',
  u'1061933',
  u'Tess Rooney',
  u'2715857',
  u'Shonali Pal',
  1],
 [44380360,
  u'2015-03-08T20:26:33Z',
  u'\U0001f373',
  u'1061933',
  u'Tess Rooney',
  u'2715857',
  u'Shonali Pal',
  1],
 [44297186,
  u'2015-03-08T06:07:32Z',
  u'\U0001f41e',
  u'1061933',
  u'Tess Rooney',
  u'1727555',
  u'Mika Fooksa',
  2],
 [44296901,
  u'2015-03-08T06:05:33Z',
  u'\U0001f35e',
  u'1061933',
  u'Tess Rooney',
  u'1634561',
  u'Lauren Bearden',
  0],
 [44296672,
  u'2015-03-08T06:04:09Z',
  u'\U0001f41b',
  u'1061933',
  u'Tess Rooney',
  u'2715857',
  u'Shonali Pal',
  1],
 [44288829,
  u'2015-03-08T05:19:59Z',
  u'\U0001f35e',
  u'1061933',
  u'Tess Rooney',
  u'117602',
  u'Dan Kearney',
  2],
 [42464313,
  u'2015-02-27T03:55:48Z',
  u'for thai food and TJs',
  u'114570',
  u'Theo Thompson',
  u'1061933',
  u'Tess Rooney',
  1],
 [41662633,
  u'2015-02-22T16:56:19Z',
  u'Turkey and bread from coop',
  u'117602',
  u'Dan K

In [176]:
def search_message_in_circle(message,id,degree):
    if degree ==2:
        friend_list = list(get_friend(id).union(get_2nd_friend(id)))
    elif degree == 1:
        friend_list = list(get_friend(id))
    #print friend_list
    query = {
        "size":50, 
    "query" : {

            "bool" : {
              "should" : [
                 { "terms" : {"actor.id" : friend_list}}, 
                 { "terms" : {"transactions.target.id" : friend_list}}], "minimum_should_match": 1,
            "must":  { "match" : {"message" : message}} 
              
              }
      },
      "sort": { "created_time": { "order": "desc" }
              }
        }
    res = es.search(index="venmo",doc_type="transaction", body=query)
    return res['hits']['hits']


In [177]:
search_message_in_circle('pizza','343071',2)

[{u'_id': u'AVVWm_Xo4VtsbIGJfBqa',
  u'_index': u'venmo',
  u'_score': None,
  u'_source': {u'action_links': {},
   u'actor': {u'cancelled': False,
    u'date_created': u'2015-04-24T03:31:23',
    u'external_id': u'1669989677400064716',
    u'firstname': u'Sarah',
    u'id': u'4451898',
    u'is_business': False,
    u'lastname': u'Vick',
    u'name': u'Sarah Vick',
    u'picture': u'https://venmopics.appspot.com/u/v1/n/cb65f8d1-749b-4304-924a-f10dadab01ae',
    u'username': u'SVick'},
   u'audience': u'public',
   u'comments': [],
   u'created_time': u'2016-03-06T00:12:19Z',
   u'likes': {u'count': 0, u'data': []},
   u'mentions': [],
   u'message': u'Pizza and Beer. YUM!!!',
   u'payment_id': 153052896,
   u'permalink': u'/story/56db75e3cd03c9af2246cbbb',
   u'story_id': u'56db75e3cd03c9af2246cbbb',
   u'transactions': [{u'target': {u'cancelled': False,
      u'date_created': u'2014-09-26T18:59:59',
      u'external_id': u'1518254153531392465',
      u'firstname': u'Nora',
      u'id

In [202]:
import operator
def list_user(name):

    
    body_target = {
  "query": {
         "match": { "transactions.target.name" : name }
  },
  "aggs" :{
    "number": {
      "terms":{
        "field":"transactions.target.id"
      }
    }
  }
}


    body_actor = {
  "query": {
         "match": { "actor.name" : name }
  },
  "aggs" :{
    "number": {
      "terms":{
        "field":"actor.id"
      }
    }
  }
} 
    res_target = es.search(index="venmo",doc_type="transaction", body=body_target)
    res_actor = es.search(index="venmo",doc_type="transaction", body=body_actor)
    #print res_target
    #print "===============\n\n"
    #print res_actor
    print res_target['aggregations']['number']['buckets']
    print res_actor['aggregations']['number']['buckets']
    ids = res_target['aggregations']['number']['buckets']+res_actor['aggregations']['number']['buckets']
    print ids
    id_dict = {}
    for id in ids:
        if id["key"] in id_dict:
            id_dict[id['key']] +=  id['doc_count']
        else:
            id_dict[id['key']] =  id['doc_count']
    print id_dict
    sorted_id  = sorted(id_dict.items(), key=operator.itemgetter(1),reverse=True)
    
    sorted_name = [(r1.get(x),y,) for x,y in sorted_id]
    return sorted_name

In [212]:
list_user("Jim Cole")

[{u'key': u'1105487', u'doc_count': 87}, {u'key': u'1689609', u'doc_count': 85}, {u'key': u'510313', u'doc_count': 74}, {u'key': u'505020', u'doc_count': 68}, {u'key': u'395186', u'doc_count': 56}, {u'key': u'1057704', u'doc_count': 54}, {u'key': u'373913', u'doc_count': 50}, {u'key': u'655441', u'doc_count': 49}, {u'key': u'290078', u'doc_count': 44}, {u'key': u'173971', u'doc_count': 43}]
[{u'key': u'208932', u'doc_count': 77}, {u'key': u'658875', u'doc_count': 77}, {u'key': u'311224', u'doc_count': 73}, {u'key': u'510313', u'doc_count': 70}, {u'key': u'397616', u'doc_count': 66}, {u'key': u'290078', u'doc_count': 58}, {u'key': u'1057704', u'doc_count': 57}, {u'key': u'395186', u'doc_count': 52}, {u'key': u'1070092', u'doc_count': 47}, {u'key': u'1166876', u'doc_count': 45}]
[{u'key': u'1105487', u'doc_count': 87}, {u'key': u'1689609', u'doc_count': 85}, {u'key': u'510313', u'doc_count': 74}, {u'key': u'505020', u'doc_count': 68}, {u'key': u'395186', u'doc_count': 56}, {u'key': u'105

[('Jack Cole', 144),
 ('Jim Hsiao', 111),
 ('Jim Xiao', 108),
 ('Cole Levin', 102),
 ('Cole DeCamp', 87),
 ('Tim Cole', 85),
 ('Cole McIlwraith', 77),
 ('Cole Palmer', 77),
 ('Jim Bonutto', 73),
 ('Cole Rider', 68),
 ('Jim Silva', 66),
 ('Cole Bender', 50),
 ('Elliot Cole', 49),
 ('Jim Nguyen', 47),
 ('Cole Mccracken', 45),
 ('Cole Nielsen', 43)]